# Sistema de Identificação e Recomendação.
---

# Carregando a base de dados artificial:


In [1]:
!pip install openpyxl

In [2]:
import openpyxl # This line imports the necessary library for reading xlsx files

In [7]:
from google.colab import drive
import pandas as pd
import os

In [9]:
# Após o upload, carregue as planilhas usando os nomes dos arquivos
df_base = pd.read_excel('Base.xlsx', engine='openpyxl')
df_culturas_pragas = pd.read_excel('Culturas_e_pragas.xlsx', engine='openpyxl')

In [12]:
df_base.head()

,Qual o município está localizado?,"Qual é o porte do produtor? (Pequeno, Médio, Grande)",Quais culturas você planta?,Qual cultura está sendo atacada?,Qual mês do ano estamos?,Quais sintomas você está observando nas plantas?,Qual a extensão do dano observado?,Há presença de insetos ou larvas visíveis nas plantas?,Você notou alterações no crescimento ou desenvolvimento das plantas?,Houve mudanças recentes nas condições do solo ou do clima?,DIAGNÓSTICO
0,Boa Vista,Pequeno,"Mandioca, Milho",Mandioca,Janeiro,Quais sintomas você está observando nas plantas?,Moderado,Sim,sim,Seca prolongada,Percevejo
1,Alto Alegre,Médio,Milho e soja,Milho,Abril,Folhas amareladas e com buracos,Moderado,Sim,Sim,Chuvas intensas,Lagarta-do-cartucho
2,Alto Alegre,Grande,Soja e arroz,Soja,Julho,Desenvolvimento atrofiado e folhas enroladas,Extenso,Sim,Sim,Não,Percevejo-da-soja
3,Iracema,Pequeno,"Banana, Mandioca e Melancia",Banana,Outubro,Presença de manchas escuras nos frutos,Leve,Não,Não,Seca prolongada,Broca-do-fruto
4,Caroebe,Pequeno,Mandioca e batata-doce,Mandioca,Julho,Raízes com sinais de podridão,Leve,Sim,Sim,Chuvas intensas,Larva-de-arame ou Corós


In [13]:
print(df_base.shape)

(50, 11)


In [14]:
df_culturas_pragas.head()

,Cultura,Categoria da Praga,Pragas Incluídas,Tratamento Nível 1 (Orgânico),Tratamento Nível 2 (Genérico),Tratamento Nível 3 (Agrotóxico Controlado)
0,Milho,Pragas do Solo,"Larva-de-arame, Larva-de-alfanete, Larva-Angor...",Uso de predadores naturais como ácaros e inse...,"Rotação de culturas, evitar monocultura e\n pr...",Aplicação de inseticidas específicos no solo ...
1,Milho,Lagartas,"Lagarta-elasmo, Lagarta-rosca, Lagarta-cartuch...","Uso de parasitoides, como vespas e besouros.",Remover plantas daninhas e restos culturais an...,"Aplicação de inseticidas de contato, principal..."
2,Milho,Percevejo,"Percevejo-verde, Percevejo-do-milho.",Introdução de parasitoides que atacam ovos.,Plantio em períodos menos favoráveis \nao perc...,"Pulverização de inseticidas, especialmente nas..."
3,Soja,Lagartas,"lagarta_da_soja, lagarta_falsa_me, lagarta_bra...",Uso de vespas parasitoides e de bactérias com...,Controle de plantas daninhas e rotação de cult...,Pulverização de inseticidas seletivos.
4,Arroz,Pragas Foliares e do Grão,"Pulgão-da-raiz, Percevejo-do-grão, Lagarta-da-...",Controle biológico com vespas parasitoides,Extrato de pimenta,Fipronil


In [15]:
print(df_culturas_pragas.shape)

(12, 6)


# A partir daqui trabalhando com os Embedings e com o Gradio

In [16]:
!pip install transformers
!pip install torch
!pip install transformers gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.7/56.7 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.8/319.8 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.7/94.7 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.5/447.5 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 79.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.3/73.3 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 7.5 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.24.7
    Uninstalling huggingface-hub-0.24.7:
      Successfully uninstalled huggingface-hub-0.24.7


In [18]:
import torch
from transformers import BertTokenizer, BertModel
import pandas as pd
import gradio as gr

# Inicializar o tokenizer e o modelo BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Obter lista de perguntas
perguntas = df_base.columns[:-1].tolist()  # Todas as perguntas, exceto a coluna "DIAGNÓSTICO"

# Função para calcular embeddings de uma lista de textos
def calcular_embeddings(textos):
    entradas = tokenizer(textos, padding=True, truncation=True, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**entradas)
    return outputs.last_hidden_state[:, 0, :]

# Pré-calcular os embeddings dos diagnósticos
diagnosticos = df_base['DIAGNÓSTICO'].tolist()
embeddings_diagnosticos = calcular_embeddings(diagnosticos)

# Função de diagnóstico
def diagnosticar(respostas_usuario):
    # Calcular embedding médio das respostas do usuário para as 10 perguntas
    embeddings_respostas = calcular_embeddings(respostas_usuario)
    embedding_medio_respostas = embeddings_respostas.mean(dim=0, keepdim=True)

    # Comparar o embedding médio das respostas com cada embedding de diagnóstico
    similaridades = torch.nn.functional.cosine_similarity(embedding_medio_respostas, embeddings_diagnosticos)
    indice_diagnostico = similaridades.argmax().item()
    diagnostico = df_base.loc[indice_diagnostico, 'DIAGNÓSTICO']

    # Buscar tratamentos na segunda planilha
    praga = df_culturas_pragas[df_culturas_pragas['Categoria da Praga'] == diagnostico]
    if not praga.empty:
        tratamento_nivel_1 = praga.iloc[0]['Tratamento Nível 1 (Orgânico)']
        tratamento_nivel_2 = praga.iloc[0]['Tratamento Nível 2 (Genérico)']
        tratamento_nivel_3 = praga.iloc[0]['Tratamento Nível 3 (Agrotóxico Controlado)']
    else:
        tratamento_nivel_1 = tratamento_nivel_2 = tratamento_nivel_3 = "Tratamento não encontrado"

    return diagnostico, tratamento_nivel_1, tratamento_nivel_2, tratamento_nivel_3

# Interface Gradio
def interface_gradio(*respostas_usuario):
    respostas_usuario = list(respostas_usuario)
    diagnostico, tratamento1, tratamento2, tratamento3 = diagnosticar(respostas_usuario)
    return [diagnostico, tratamento1, tratamento2, tratamento3]

inputs = [gr.Textbox(label=pergunta) for pergunta in perguntas]
outputs = [
    gr.Textbox(label="Diagnóstico"),
    gr.Textbox(label="Tratamento Nível 1 (Orgânico)"),
    gr.Textbox(label="Tratamento Nível 2 (Genérico)"),
    gr.Textbox(label="Tratamento Nível 3 (Agrotóxico Controlado)")
]

gr.Interface(
    fn=interface_gradio,
    inputs=inputs,
    outputs=outputs,
    title="Diagnóstico e Tratamento de Pragas",
    description="Responda às perguntas para obter um diagnóstico e as opções de tratamento."
).launch()


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://48737cfd35bcf8853c.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
